In [2]:
import pandas as pd
import numpy as np

In [3]:
def headtail(df):
    return pd.concat([df.head(), df.tail()])

### Чтение датасета: https://www.kaggle.com/datasets/sharthz23/recsys-yandex-cup-2022

In [4]:
df = pd.read_csv('../input/recsys-yandex-cup-2022/dataframe.csv/dataframe.csv')
# Переопределение user_id
max_train_user_id = df[df.is_train == 1].user_id.max()
df.loc[df.is_train == 0, "user_id"] += max_train_user_id
# Модерация атрибутивного состава
df = df[['user_id', 'track_id', 'reversed_rank']]
df = df.rename(columns={"user_id": "user", "track_id": "item", "reversed_rank": "order"})
headtail(df)

,user,item,order
0,0,333396,53
1,0,267089,52
2,0,155959,51
3,0,353335,50
4,0,414000,49
117450829,1449996,448288,4
117450830,1449996,1343,3
117450831,1449996,86420,2
117450832,1449996,186436,1
117450833,1449996,8474,0


### Класс, реализующий схему валидации UsersKFoldLeavePOut
**Решаемая задача** - построить схему валидации для данного соревнования с учетом особенностей сорвенования
- Между `train` и `test` не должно быть общих пользователей
- Количество фолдов задается через параметр класса `n_folds`
- В `test` должно быть не более `p` последних треков (параметр класса `p`)

In [7]:
class UsersKFoldPOut():
    def __init__(self, n_folds, p, random_seed=23):
        self.n_folds = n_folds
        self.p = p
        self.random_seed = random_seed
    
    def split(self, df):
        users = df.user.unique()
        users_count = len(users)
        
        np.random.seed(self.random_seed)
        np.random.shuffle(users)
        
        fold_sizes = np.full(self.n_folds, users_count // self.n_folds, dtype=int)
        fold_sizes[: users_count % self.n_folds] += 1
        current = 0
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            test_fold_users = users[start:stop]
            # Для теста берутся первые p взаимодействий пользователей, попавших в тестовую часть фолда
            test_mask = (df.user.isin(test_fold_users)) & (df.order < self.p)
            # Для трейна берутся все взаимодействия пользователей, не попавших в тестовую часть фолда
            train_mask = ~df.user.isin(test_fold_users)
            
            yield train_mask, test_mask

### Демонстрация работы для трёх фолдов и аута по одному взаимодействию

In [8]:
cv = UsersKFoldPOut(n_folds=3, p=1)

for i, (train_mask, test_mask) in enumerate(cv.split(df)):
    train = df[train_mask]
    test = df[test_mask]
    print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')

Fold#0 | Train: 78193062, Test: 483333
Fold#1 | Train: 78193081, Test: 483332
Fold#2 | Train: 78193081, Test: 483332
